In [ ]:
import time
import folium
import geopandas as gpd #A more flexible package to work with geospatial data in python 
import json
import matplotlib.pyplot as plt
import networkx as nx # networkx package 
import osmnx as ox 
import pandas as pd #Base package for data analysis and manipulation 
from pyproj import CRS #package for your projection management 
import random
import rasterio as rio
import rasterstats as rs
import requests
import shapely.geometry #python package for basic spatial operation 

### INTRODUCTION

The sections of this notebook are in order. 
The OSM settings should always be run.
Some of the operations in this notebook require data that can be downloaded from the data folder in this repository.

Downloading the AHN4 datasets can take quite some time, so make sure you have everything set up right before you do that. This also counts for downloading the networks.

### OSMnx settings
The settings of OSMnx was changed with the settings module to extend the list of tags needed for the analysis

In [ ]:
ox.settings.useful_tags_way.extend(['surface','footway','cycleway'])
ox.settings.useful_tags_node.extend(['barrier'])

### NETWERK EXTENT 

In [ ]:
g_namen = ['Amsterdam', 'Den_Haag', 'Rotterdam', 'Utrecht']#set the city names
project_gemeenten = gpd.read_file(r"data\boundaries\UrbanRunner_Areas.geojson") #download city geometries from file
project_gemeenten["extent"] = project_gemeenten.envelope.buffer(2000).envelope.to_crs("EPSG:4326") #set additional geometry column of extent
project_gemeenten.set_index('gemeentenaam', inplace=True) #set index to gemeentenaam column
project_gemeenten.to_crs("EPSG:4326", inplace=True) #project to WGS84

##### plot in folium

In [ ]:
map_centroid = project_gemeenten.unary_union.envelope.centroid.coords[0] #point focus for folium map
m = folium.Map(location = (map_centroid[1], map_centroid[0])) #create folium map
#plot the extens of each network region in the folium map
for _, r in project_gemeenten.iterrows():
    #unbuffered convex hull
    extent = gpd.GeoSeries(r["extent"])
    extent_j = extent.to_json()
    extent_j = folium.GeoJson(data=extent_j, style_function=lambda x: {'fillColor': 'blue'})
    folium.Popup(r.name).add_to(extent_j)
    extent_j.add_to(m)

    #orginal geometry
    geom = gpd.GeoSeries(r['geometry'])
    geom_j = geom.to_json()
    geom_j = folium.GeoJson(data=geom_j, style_function=lambda x: {'fillColor': 'green'})
    folium.Popup(r.name).add_to(geom_j)
    geom_j.add_to(m)

m

### GETTING NETWORK

In [ ]:
#download network for first time
for g in g_namen: ## beschrijven dat er twee netwerken worden gepakt: van walk en van bike, en dat deze dan worden samengevoegd met de networkx compose functie. 
    network_walk = ox.graph_from_polygon(project_gemeenten.loc[g].extent, network_type="walk", retain_all=True ,simplify=False) ## we hebben gekozen voor simplify false. Beschrijven wat dit inhoudt ahv osmnx documentation. We hebben de dijkstra algoritme getest op twee netwerken, één met simplify aan en de ander met simplify uit. Er was wel een verschil, maar niet groot genoeg om simplify te gebruiken. Retain_all= true moet ook worden uitgelegd waarom. Dit is namelijk omdat we twee verschillende netwerken samenvoegen, en dat we daarna de niet verbonden edges er afgooien.
    network_bike = ox.graph_from_polygon(project_gemeenten.loc[g].extent, network_type="bike", retain_all=True, simplify=False)
    network_both = nx.compose(network_walk, network_bike)
    globals()[f"{g}_network_both"] = ox.utils_graph.get_largest_component(network_both)
    print('finished network of', g)

### DOWNLOADING AHN, ADDING NODE ELEVATION, ADDING EDGE GRADES

#### DOWNLOADING AHN

In [ ]:
ahn_datavlakken = gpd.read_file(r"data/kaartbladen_AHN4.gpkg") #this is a spatial layers of vector tiles. Each vector tile contains download links of the AHN products for that specific tile.
ahn_datavlakken.to_crs("EPSG:4326", inplace=True)
for g in g_namen:
    gemeente_AHN_datavlakken = ahn_datavlakken.clip(project_gemeenten.loc[g].extent)
    for i in range(globals()[f"{g}_ahnvlakken"].Name_1.count()):    
        name = globals()[f"{g}_ahnvlakken"].iat[i,2]
        response = requests.get(globals()[f"{g}_ahnvlakken"].iat[i,3])
        print('done with downloading number ' + i + ' of gemeente ' + g)
        open('data\\AHN4_05M_DTM\\'+g + '\\'+ name + '.zip', "wb").write(response.content)
# list of paths to the different DTM tiles is made
    globals()[f"{g}_DTM_list"] = []
    for i in range(gemeente_AHN_datavlakken.Name_1.count()):    
        name = gemeente_AHN_datavlakken.iat[i,2]
        globals()[f"{g}_DTM_list"].append('data\\AHN4_05M_DTM\\'+ g + '\\M_'+ name +".tif")

##### the above code results in the AHN products to be downloaded as zip. For our project, these were extracted manually

In [ ]:
for g in g_namen:
    globals()[f"{g}_network_both"] = ox.project_graph(globals()[f"{g}_network_both"], to_crs="epsg:28992")
    globals()[f"{g}_network_both"] = ox.add_node_elevations_raster(globals()[f"{g}_network_both"], globals()[f"{g}_DTM_list"])
    globals()[f"{g}_nodes"], globals()[f"{g}_edges"] = ox.graph_to_gdfs(globals()[f"{g}_network_both"])

### ADDING HEAT DATA

In [ ]:
def EnrichEdgesWithRasterInfo(edges, raster, statsprefix):
    edges['UID'] = range(0, len(edges)) # add a temporary Unique ID column
    e = edges.loc[:, ['UID', 'geometry']] # make subset of relevant columns
    e_zonalstats = rs.zonal_stats(e, raster, prefix=statsprefix, geojson_out=True) # perform spatial overlay/ zonal statistics and add statistics
    e_props = pd.DataFrame.from_dict(e_zonalstats).properties # convert to dataframe and select only the properties (results again in dictionary)
    e_propsdf = pd.DataFrame.from_dict(list(e_props)) # convert dictionary with properties to a pandas dataframe
    edges_updated = edges.join(other=e_propsdf.set_index('UID'), on=('UID')) # join stats to the original edges and save as updated edges
    return edges_updated

In [ ]:
# Preprocess UHIi map
rastfile = rio.open('data/RIVM_R88_20170621_gm_actueelUHI.tif')
for g in g_namen:   
    bboxshape = [json.loads(gpd.GeoDataFrame({"geometry": project_gemeenten.loc[g].extent.buffer(0.1).envelope}, 
        index=[0], crs="EPSG:4326").to_crs("EPSG:28992").to_json())['features'][0]['geometry']]
    UHImap, UHImap_transform = rio.mask.mask(rastfile, shapes=bboxshape, crop=True)
    with rio.open(f"{g}_hittedata.tif", 'w', driver="GTiff",
                   height=UHImap.shape[1], width=UHImap.shape[2], 
                   transform=UHImap_transform, crs=CRS.from_epsg(28992),
                   nodata=255, dtype='float32', count= 1) as file:
        file.write(UHImap)

In [ ]:
#PLOTTING MAP
UHI = rio.open('utrecht_hittedata.tif').read(1, masked=True)
fig, ax = plt.subplots(figsize=(20,20))
UHIplot = ax.imshow(UHI, cmap='jet')
ax.set_title("UHI map of area", fontsize=25)
cbar = fig.colorbar(UHIplot, fraction=0.035, pad=0.01)
cbar.ax.get_yaxis().labelpad = 15
cbar.ax.set_ylabel('physiologocal equivalent temperature (°C)', rotation=270)
ax.set_axis_off()
plt.show()

In [ ]:
for g in g_namen:
    globals()[f"{g}_edges"] = EnrichEdgesWithRasterInfo(globals()[f"{g}_edges"], f"{g}_hittedata.tif", 'UHI_')

### FINISHING SLOPE DATA

#### DOWNLOADING NETWORK AND REGRAPHING

The nodes are downloaded to bring into QGIS. In QGIS the nodes with the missing elevation is interpolated.

In [ ]:
for g in g_namen:
    globals()[f"{g}_nodes"].to_file(f'data/graphs/retry/{g}_graph.gpkg', driver = "GPKG", layer= 'nodes') 
    globals()[f"{g}_edges"].to_file(f'data/graphs/retry/{g}_graph.gpkg', driver = "GPKG", layer= 'edges') 

In QGIS the following steps are performed:
1. add nodes layer;
2. in field calculator run the expression for the added node layer:

        if(elevation is Null, array_mean(overlay_nearest(layer:='nodes', filter:= elevation is not Null, expression:= elevation, limit:=5), elevation)
3. save edits.



##### regraph, add edge grades, and get nodes and edges gdfs

In [ ]:
for g in g_namen:
    globals()[f"{g}_nodes"] = gpd.read_file(f'data/graphs/retry/{g}_graph.gpkg', layer= 'nodes').set_index('osmid')
    globals()[f"{g}_edges"] = gpd.read_file(f'data/graphs/retry/{g}_graph.gpkg', layer= 'edges').set_index(['u','v','key'])
    globals()[f"{g}_network_both"] = ox.graph_from_gdfs(globals()[f"{g}_nodes"],globals()[f"{g}_edges"])
    globals()[f"{g}_network_both"] = ox.add_edge_grades(globals()[f"{g}_network_both"])
    globals()[f"{g}_nodes"], globals()[f"{g}_edges"] = ox.graph_to_gdfs(globals()[f"{g}_network_both"], nodes=True, edges=True, node_geometry=False, fill_edge_geometry=False)

### GREENSPACES, WATERTAPS AND OBSTACLES

In [ ]:
for g in g_namen:
    globals()[f"greenspaces_{g}"] = ox.geometries_from_polygon(project_gemeenten.loc[g].extent, tags={'landuse':['village_green','recreation_ground','grass','forest'], 'leisure':['nature_reserve','park','garden'],'natural':['fell','heath','wood','wetland','coastline','beach']}) ## beschrijven
    globals()[f"greenspaces_{g}"] = globals()[f"greenspaces_{g}"].loc['way'][["geometry","natural","leisure","landuse"]].to_crs("EPSG:28992")
    globals()[f"greenspaces_{g}"].geometry = globals()[f"greenspaces_{g}"].buffer(5)

    globals()[f"waterpoints_{g}"] = ox.geometries_from_polygon(project_gemeenten.loc[g].extent, tags={'amenity':'drinking_water'})
    globals()[f"waterpoints_{g}"].to_crs('EPSG:28992', inplace=True)
    globals()[f"waterpoints_{g}"]['buffered'] = globals()[f"waterpoints_{g}"].buffer(50) ## dit moet worden onderbouwd: waarom 50 meter buffer? & in de discussie: watertappunten is lastig, want je wilt niet na 100m al langs een watertappunt lopen
    globals()[f"waterpoints_{g}"].set_geometry('buffered', inplace=True, crs="EPSG:28992")
    globals()[f"waterpoints_{g}"] = globals()[f"waterpoints_{g}"][['amenity','buffered']]

##### adding attributes to edges

In [ ]:
for g in g_namen:
    globals()[f"{g}_edges"] = gpd.sjoin(globals()[f"{g}_edges"], globals()[f"greenspaces_{g}"], rsuffix = "greenspace", how = 'left')
    globals()[f"{g}_edges"] = gpd.sjoin(globals()[f"{g}_edges"], globals()[f"waterpoints_{g}"], rsuffix = "waterpoints", how = 'left')
    nodes_disruptions = globals()[f"{g}_nodes"][['highway', 'barrier']]
    globals()[f"{g}_edges"] = globals()[f"{g}_edges"].join(nodes_disruptions.rename_axis('u'), how = 'left', rsuffix = "_obstnodes_u")
    globals()[f"{g}_edges"] = globals()[f"{g}_edges"].join(nodes_disruptions.rename_axis('v'), how = 'left', rsuffix = "_obstnodes_v")


### CHECKING THE RESULTING NETWORK

##### CHECKING UNIQUE VALUE

In [ ]:
attributes = ['UHI_mean', 'barrier', 'highway', 'cycleway', 'footway', 'surface', 'highway_obstnodes_u','barrier']
for v in attributes:
    globals()[f"{v}_uniques"] = []
    for g in g_namen:
        globals()[f"{v}_uniques"].extend(globals()[f"{g}_edges"][v].unique())
    globals()[f"{v}_uniques"] = list(set(globals()[f"{v}_uniques"]))

##### checking occurences of values

In [ ]:
def occurence_value(value, variable):
    for g in g_namen:
        edges = globals()[f"{g}_edges"]
        print(value, 'occurs', len(edges[edges[variable] == value]), 'times for the variable', variable, 'in the area of', g)

In [ ]:
occurence_value('asphalt', 'surface')

### COST FUNCTION

##### Standardizations

In [ ]:
for g in g_namen:
    edges = globals()[f"{g}_edges"] 
    #waterpoints
    edges['cost_waterpoints'] = 1
    edges['cost_waterpoints'][edges.amenity == 'drinking_water'] = 0
    #UHI
    edges['cost_UHI'][edges['UHI_mean'] < 0] == 0 
    edges['cost_UHI'] = edges['UHI_mean'] / 3 #3 comes from the highest UHI_mean value of all the areas
    #Greenspace
    edges['cost_greenspace'] = 0
    edges['cost_greenspace'][(edges['landuse'].isnull()) & (edges['leisure'].isnull()) & (edges['natural'].isnull())] = 1
    #slope
    edges['cost_grade'] = 0
    edges['cost_grade'][(edges['grade'] > 0)] = edges['grade'] / 5.982 #5.982 is the highest grade value of all the areas
    #disruptions
    edges['cost_disruptions'] = 0
    edges['cost_disruptions'][(edges['footway'] == 'crossing') |(edges['cycleway'] == 'crossing') |(edges['barrier_obstnodes_u'] == 'gate')|(edges['barrier_obstnodes_u'] == 'wicket_gate')|(edges['barrier_obstnodes_u'] == 'stile')|(edges['barrier_obstnodes_u'] == 'turnstile')|(edges['barrier_obstnodes_u'] == 'full-height_turnstile')] = 0.5
    edges['cost_disruptions'][(edges['highway_obstnodes_u'] == 'traffic_signals')|(edges['highway_obstnodes_v'] == 'traffic_signals')] = 1
    #surface_unpaved
    edges['cost_surface_prefunpaved'] = 0.5
    edges['cost_surface_prefunpaved'][edges['surface'].isin(['dirt;grass','grass_paver','mud','dirt','unpaved','wetland','clay','grass','sand','ground','earth','woodchips','reed_bed','soil'])] = 0
    edges['cost_surface_prefunpaved'][edges['surface'].isin(['dirt;gravel','crushed_shells','gravel','fine_gravel','shells','schelpen','rubber'])] = 0.25
    edges['cost_surface_prefunpaved'][edges['surface'].isin(['compacted','wood','resin','rasin'])] = 0.75
    edges['cost_surface_prefunpaved'][edges['surface'].isin(['concrete-slabs','steel','paving_stones;asphalt','concrete:lanes','concrete_slab','concrete_slabs','concrete:deconstructed_czech_hedgehogs','paved','asphalt','metal','marble','concrete:plates','concrete:tiles','concrete','zinc','bricks','brick','tiles','concrete_tiles:30','paving_stones:20','paving_stones:30','metal_grid','asphalt;paving_stones:30x30','stepping_stones','paving_stones','stone','pebblestone','cobblestone','unhewn_cobblestone'])] = 1
    #surface_paved
    edges['cost_surface_prefpaved'] = 0.5
    edges['cost_surface_prefpaved'][edges['surface'].isin(['dirt;grass','grass_paver','mud','dirt','unpaved','wetland','clay','grass','sand','ground','earth','woodchips','reed_bed','soil'])] = 1
    edges['cost_surface_prefpaved'][edges['surface'].isin(['dirt;gravel','crushed_shells','gravel','fine_gravel','shells','schelpen','rubber'])] = 0.75
    edges['cost_surface_prefpaved'][edges['surface'].isin(['compacted','wood','resin','rasin'])] = 0.25
    edges['cost_surface_prefpaved'][edges['surface'].isin(['concrete-slabs','steel','paving_stones;asphalt','concrete:lanes','concrete_slab','concrete_slabs','concrete:deconstructed_czech_hedgehogs','paved','asphalt','metal','marble','concrete:plates','concrete:tiles','concrete','zinc','bricks','brick','tiles','concrete_tiles:30','paving_stones:20','paving_stones:30','metal_grid','asphalt;paving_stones:30x30','stepping_stones','paving_stones','stone','pebblestone','cobblestone','unhewn_cobblestone'])] = 0
        
    globals()[f"{g}_edges"] = edges
    

##### CALCULATING FINAL COST COLUMN

In [ ]:
for g in g_namen:
    edges = globals()[f"{g}_edges"]
    edges['final_cost_willemijn'] = ((0.7*edges['cost_UHI']) + (0.9*edges['cost_greenspace']) + (0.5*edges['cost_grade']) + (0.1*edges['cost_disruptions']) + (0.3*edges['cost_surface_prefpaved'])) * edges['length']
    edges['final_cost_rachid'] = ((0.5*edges['cost_waterpoints']) + (0.7*edges['cost_UHI']) + (0.9*edges['cost_greenspace']) + (0.3*edges['cost_disruptions']) + (0.1*edges['cost_surface_prefunpaved'])) * edges['length']
    globals()[f"{g}_edges"] = edges 

##### SAVE GRAPHS

In [ ]:
for g in g_namen:
    globals()[f"{g}_edges"].to_file(f'data/graphs/retry/with_costs/{g}_edges_with_costs.gpkg', driver = "GPKG", layer= 'edges')

## TESTING

In [ ]:
Utrecht_centrum_extent = gpd.GeoSeries(
    shapely.geometry.Polygon([(135874.0,456545.4), (137381.2 ,456742.0), (137459.9 ,454749.8), (136018.1, 454703.9)]).envelope, 
    crs= "EPSG:28992").to_crs("epsg:4326") #small part of Utrecht is chosen as


##### Getting network

In [ ]:
# simplify True
Utrecht_network_walk = ox.graph_from_polygon(Utrecht_centrum_extent[0], network_type="walk")
Utrecht_network_bike = ox.graph_from_polygon(Utrecht_centrum_extent[0], network_type="bike")
Utrecht_network_both_test_simplified = nx.compose(Utrecht_network_walk, Utrecht_network_bike)

#simply false
Utrecht_network_walk = ox.graph_from_polygon(Utrecht_centrum_extent[0], network_type="walk", simplify=False)
Utrecht_network_bike = ox.graph_from_polygon(Utrecht_centrum_extent[0], network_type="bike", simplify=False)
Utrecht_network_both_test_unsimplified = nx.compose(Utrecht_network_walk, Utrecht_network_bike)

In [ ]:
Utrecht_centrum_extent[0]

##### Routing

In [ ]:
def get_random_XY_in_polygon(poly): ## manier om XY coordinaten te pakken binnen een polygon
    minx, miny, maxx, maxy = poly.bounds
    while True:
        p = shapely.geometry.Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
        if poly.contains(p):
            return p.coords[0][0], p.coords[0][1]

In [ ]:
X_list = []
Y_list = []    
for i in range(100):
    x, y = get_random_XY_in_polygon(Utrecht_centrum_extent[0])
    X_list.append(x)
    Y_list.append(y)

#getting nodes from the random XY sets
#UNSIMPLIFIED
Utrecht_random_nodes_unsimplified = ox.nearest_nodes(Utrecht_network_both_test_unsimplified, X_list, Y_list, return_dist=True)
Utrecht_source_nodes_unsimplified = Utrecht_random_nodes_unsimplified[0][:50]
Utrecht_goal_nodes_unsimplified   = Utrecht_random_nodes_unsimplified[0][50:100]
#SIMPLIFIED
Utrecht_random_nodes_simplified = ox.nearest_nodes(Utrecht_network_both_test_simplified, X_list, Y_list, return_dist=True)
Utrecht_source_nodes_simplified = Utrecht_random_nodes_simplified[0][:50]
Utrecht_goal_nodes_simplified   =   Utrecht_random_nodes_simplified[0][50:100]

In [ ]:
# calculating shortest route of the source and goal nodes for both simplified and unsimplified network, then comparing computing time
#UNSIMPLIFIED
start_time = time.time()
random_shortest_routes = ox.distance.shortest_path(
    Utrecht_network_both_test_unsimplified, Utrecht_source_nodes_unsimplified, Utrecht_goal_nodes_unsimplified, weight='length', cpus=1)

print("gathering random routes for unsimplified network takes %s seconds" % (time.time() - start_time))
randomroute_time_unsimplified = (time.time() - start_time)

#SIMPLIFIED
start_time = time.time()
random_shortest_routes_simplified = ox.distance.shortest_path(
    Utrecht_network_both_test_simplified, Utrecht_source_nodes_simplified, Utrecht_goal_nodes_simplified, weight='length', cpus=1)
    
print("gathering random routes for simplified network takes %s seconds" % (time.time() - start_time))
randomroute_time_simplified = (time.time() - start_time)